### ***Data Preparation***

In [44]:
import pandas as pd 
import warnings 
import numpy as np 
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import linregress


def calculate_dewpoint(temp, humidity):
    A = 17.27
    B = 237.7
    alpha = ((A * temp) / (B + temp)) + np.log(humidity/100.0)
    return (B * alpha) / (A - alpha)

def reading_data(pr_dir, tas_dir, hur_dir, start_year, stop_year):
    years_list = [str(year) for year in range(start_year, stop_year)]
    
    interp_pr, interp_tas, interp_hur, interp_tdew = {}, {}, {}, {}

    for year in years_list: 
        file_name = f"{year}.csv"
        # pr
        file_pr = pd.read_csv(pr_dir + "\\" + file_name)
        file_pr = file_pr[file_pr.lat > -60]
        # tas
        file_tas = pd.read_csv(tas_dir+ "\\" + file_name)
        file_tas = file_tas[file_tas.lat > -60]
        # hur
        file_hur = pd.read_csv(hur_dir+ "\\" + file_name)
        file_hur = file_hur[file_hur.lat > -60]
        interp_pr[year], interp_tas[year], interp_hur[year] = file_pr, file_tas, file_hur 
    
    # Calculate Dewpoint temperature 

    warnings.filterwarnings(action='ignore')

    interp_tdew = {}

    for key in interp_tas.keys():
        tas = interp_tas[key]
        hur = interp_hur[key]
        tdew = pd.DataFrame()
        tdew[["lat", "lon"]] = tas[["lat", "lon"]]
        for day in tas.columns[2:]:
            tdew[day] = calculate_dewpoint(tas[day], hur[day])
        interp_tdew[key] = tdew 
    
    # Combine pr and tdew 
    pr = pd.concat([df.set_index(["lat", "lon"]) for df in list(interp_pr.values())], axis=1).reset_index()
    pr['lat_lon'] = pr['lat'].astype(str) + ',' + pr['lon'].astype(str)
    pr = pr.drop(['lat', 'lon'], axis=1)
    pr_long = pd.melt(pr, id_vars=['lat_lon'], var_name='date', value_name='pr')
    tdew = pd.concat([df.set_index(["lat", "lon"]) for df in list(interp_tas.values())], axis=1).reset_index()
    tdew['lat_lon'] = tdew['lat'].astype(str) + ',' +  tdew['lon'].astype(str)
    tdew = tdew.drop(['lat', 'lon'], axis=1)
    tdew_long = pd.melt(tdew, id_vars=['lat_lon'], var_name='date', value_name='tdew')
    tdew_values = tdew_long['tdew'].values
    truncated_values = tdew_values[:len(pr_long.index)]
    # Assign the truncated values to the DataFrame column
    pr_long["tdew"] = truncated_values
   
    pr_long = pr_long[pr_long.pr >= 0.1]
    
    return pr_long 

#### ***Quantiel Regression***

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from concurrent.futures import ThreadPoolExecutor

def fit_quantile_regression(group, q):
    model = sm.QuantReg(np.log(group['pr']), sm.add_constant(group['tdew']))
    result = model.fit(q=q)
    return result.params['tdew']

def quantile_regression(df, q):
    grouped = df.groupby('lat_lon')
    
    with ThreadPoolExecutor() as executor:
        slopes = list(executor.map(lambda group: fit_quantile_regression(group, q), [group for name, group in grouped]))

    slope_df = pd.DataFrame({
        'lat_lon': grouped.groups.keys(),
        'slope': slopes
    })
    
    slope_df['scaling'] = 100 * (np.exp(slope_df['slope']) - 1)

    print("done --> qr")
    
    return slope_df


#### ***Binning with equal bin width***

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from collections import defaultdict

def binning_w(df, q):

    tdew_min = df['tdew'].min()
    tdew_max = df['tdew'].max()
    bin_width = 2
    tdew_bins = np.arange(tdew_min, tdew_max + bin_width, bin_width)
    
    df['tdew_bin'] = df.groupby('lat_lon')['tdew'].transform(lambda x: np.digitize(x, bins=tdew_bins))
    
    df_avg_tdew = df.groupby(['lat_lon', 'tdew_bin'])['tdew'].mean().reset_index()
    df_avg_tdew.rename(columns={'tdew': 'avg_tdew'}, inplace=True)
    
    df_percentiles_pr = df.groupby(['lat_lon', 'tdew_bin'])['pr'].quantile(q).reset_index()
    df_percentiles_pr.rename(columns={'pr': '99th_percentile_pr'}, inplace=True)

    df_final = df_avg_tdew.merge(df_percentiles_pr, on=['lat_lon', 'tdew_bin'], how='left')

    coefficients = defaultdict(float)
    for lat_lon, group in df_final.groupby('lat_lon'):
        X = group['avg_tdew'].values.reshape(-1, 1)
        y = group['99th_percentile_pr'].values
    
        model = LinearRegression()
        model.fit(X, np.log(y))
    
        slope = model.coef_[0]  
        coefficients[lat_lon] = slope

    result_df = pd.DataFrame(list(coefficients.items()), columns=['lat_lon', 'slope'])
    result_df["scaling"] = 100 * (np.exp(result_df['slope']) - 1)
    result_df[['lat', 'lon']] = result_df['lat_lon'].str.split(',', expand=True)
    result_df['lat'] = result_df['lat'].astype(float)
    result_df['lon'] = result_df['lon'].astype(float)
    result_df.drop(columns=['lat_lon'], inplace=True)

    print("done --> binning_w")

    return result_df

#### ***Binning with Equal Sample Data***

In [4]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.linear_model import LinearRegression

def binning_p(df, q, loess_frac=0.1):
    def _calculate_stats(group):
        group['pr_bin'] = pd.qcut(group['tdew'], q=30, labels=False, duplicates='drop')
        group['p99_pr'] = group.groupby('pr_bin')['pr'].transform(lambda x: x.quantile(q))
        group['avg_tdew'] = group.groupby('pr_bin')['tdew'].transform('mean')
        return group[['lat_lon', 'pr_bin', 'p99_pr', 'avg_tdew']].drop_duplicates()

    def has_peak_point(x, y):
        diff_sign = np.sign(np.diff(y))
        if np.all(diff_sign[:-1] == diff_sign[1:]):
            return False
        else:
            return True

    def fit_loess(df, frac):
        smoothed_dfs = []

        for lat_lon, group in df.groupby('lat_lon'):
            smoothed = lowess(group['p99_pr'], group['avg_tdew'], frac=frac, return_sorted=False)
            smoothed_df = pd.DataFrame({
                'lat_lon': lat_lon,
                'avg_tdew': group['avg_tdew'],
                'p99_pr_smooth': smoothed
            })
            smoothed_dfs.append(smoothed_df)

        smoothed_df = pd.concat(smoothed_dfs, ignore_index=True)

        return smoothed_df
    
    def detect_peak_points(df):
        peak_points = []

        for lat_lon, group in df.groupby('lat_lon'):
            if has_peak_point(group['avg_tdew'], group['p99_pr_smooth']):
                peak_point_idx = np.argmax(group['p99_pr_smooth'])
                peak_point = group.iloc[peak_point_idx]
                peak_points.append(peak_point)

        peak_points_df = pd.DataFrame(peak_points)

        return peak_points_df

    def fit_linear_regression(df, peak_points_df):
        regression_dfs = []

        for lat_lon, group in df.groupby('lat_lon'):
            if lat_lon in peak_points_df['lat_lon'].values:
                peak_point_idx = peak_points_df[peak_points_df['lat_lon'] == lat_lon].index[0]
                group_before_peak = group.iloc[:peak_point_idx]
                if peak_point_idx == 0:
                    group_before_peak = group
                else:
                    group_before_peak = group.iloc[:peak_point_idx]
            else:
                group_before_peak = group

            X = group_before_peak[['avg_tdew']]
            y = np.log(group_before_peak['p99_pr_smooth'].abs())

            if y.isnull().any():
                group_before_peak = group_before_peak.dropna(subset=['p99_pr_smooth'])
                X = group_before_peak[['avg_tdew']]
                y = np.log(group_before_peak['p99_pr_smooth'].abs())

            model = LinearRegression().fit(X, y)
            slope = model.coef_[0]
            intercept = model.intercept_

            regression_df = pd.DataFrame({
                'lat_lon': lat_lon,
                'slope': slope,
                'intercept': intercept
            }, index=[0])

            regression_dfs.append(regression_df)

        regression_df = pd.concat(regression_dfs, ignore_index=True)
        regression_df["scaling"] = 100*(np.exp(regression_df["slope"])-1)
        return regression_df
    
  
    result_df = df.groupby('lat_lon').apply(_calculate_stats).reset_index(drop=True)
    
    smoothed_df = fit_loess(result_df, frac=loess_frac)
    
    peak_points_df = detect_peak_points(smoothed_df)
    
    regression_df = fit_linear_regression(smoothed_df, peak_points_df)
    
    print("done --> binning_p")
    return regression_df


#### ***Apply Functions***

***Noted: file_name here refers to location a csv file which is a location of files in each year for each variables, in each models/scenarios***

In [35]:
file_dir = pd.read_csv(r"file_name.csv")

In [45]:
for file in range(len(file_dir)):
    model = file_dir["Model"][file]
    scenario = file_dir["Scenario"][file]
    pr_dir = file_dir["Pr"][file]
    tas_dir = file_dir["Tas"][file]
    hurs_dir = file_dir["Hurs"][file]

    data = reading_data(pr_dir, tas_dir, hurs_dir, start_year = 2041, stop_year = 2091)
    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    data = data.dropna(subset=['date'])
    data["year"] = data['date'].dt.year
    data.drop('date', axis = 1, inplace = True)

    df = data.copy()

    print('---------------', model , 'x ', scenario, '---------------')
    qr99 = quantile_regression(df, 0.99)
    qr99.to_csv(r"folder_to_save\\q99_qr_whole_grid_" + model + "-" + scenario + ".csv")
         
    qr95 = quantile_regression(df, 0.95)
    qr95.to_csv(r"folder_to_save\\q95_qr_whole_grid_" + model + "-" + scenario + ".csv")
         
    bw99 = binning_w(df, 0.99)
    bw99.to_csv(r"folder_to_save\\q99_binning-w_whole_grid_" + model + "-" + scenario + ".csv")
         
    bw95 = binning_w(df, 0.95)
    bw95.to_csv(r"folder_to_save\\q95_binning-w_whole_grid_" + model + "-" + scenario + ".csv")
         
    bp99 = binning_p(df, 0.99)
    bp99.to_csv(r"folder_to_save\\q99_binning-p_whole_grid_" + model + "-" + scenario + ".csv")
         
    bp95 = binning_p(df, 0.95)
    bp95.to_csv(r"folder_to_save\\q95_binning-p_whole_grid_" + model + "-" + scenario + ".csv")
   

--------------- hadgem2-es x  rcp85 ---------------
done --> qr
done --> qr
done --> binning_w
done --> binning_w
done --> binning_p
done --> binning_p
